# Transform_paragraph_indents

A partir da função _transform_legislations o texto será retornado com os tokens de recuo criados.

Essa função padroniza as chamadas de alineas, identificando-as implicitamente caso sejam chamados após um parágrafo, de forma que:

- ST_PARAGRAFO_UNICO, "a" seja substituído por ST_PARAGRAFO_UNICO, ST_ALINEA_A

In [5]:
import re
ST_PREFIX = "ST_"

def _transform_legislations(
    text,
    prefix_result,
    re_legislation,
    is_multiple=False,
    identification=None,
    previous_token=None,
    separator=None,
):
    "Create custom tokens for legislation"

    def build_token(m):
        if m.group("ignored_expression"):
            result = m.group("ignored_expression")
        else:
            result = ""
        result = (
            result
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    def build_token_multi(m):
        result = (
            m.group("ignored_expression")
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    # replaces first occurrences
    text = re_legislation.sub(build_token, text)

    # replaces multiple occurrences, if exists
    if is_multiple:
        regex_legislation_multi = re.compile(
            fr"(?:(?P<ignored_expression>{previous_token}{separator}){identification})",
            flags=re.I,
        )
        while regex_legislation_multi.findall(text) != []:
            text = regex_legislation_multi.sub(build_token_multi, text)

    return text


In [2]:
def transform_paragraph_indents(text, **kwargs):
    #Create custom tokens for indents contained in a paragraph and not explicitly mentioned in the text

    prefix_result = "ALINEA_"
    previous_token = fr"{ST_PREFIX}{prefix_result}\w+"
    separator = r"\s*[,e]\s+"
    indent = (
        r"(?<!\w)[\"|\“|\”]?(?P<identification>[a|b]?[a-z])[\"|\“|\”]?(?!\w)"
    )
    re_legislation = re.compile(
        fr"(?:(?P<ignored_expression>{ST_PREFIX}PARAGRAFO_(\d+|UNICO),\s+){indent})",
        flags=re.I,
    )

    return _transform_legislations(
        text,
        prefix_result,
        re_legislation,
        True,
        indent,
        previous_token,
        separator,
    )


In [3]:
entradas = """
  ST_PARAGRAFO_UNICO, "a"
  ST_PARAGRAFO_UNICO, "a", "b"
  ST_PARAGRAFO_3, a, b, c e e, do art. 102
  art. 102, ST_PARAGRAFO_2, “a” da Constituição Federal
"""

print("Entrada: ", entradas)

Entrada:  
  ST_PARAGRAFO_UNICO, "a"
  ST_PARAGRAFO_UNICO, "a", "b"
  ST_PARAGRAFO_3, a, b, c e e, do art. 102
  art. 102, ST_PARAGRAFO_2, “a” da Constituição Federal



In [6]:
print("Saida: ",transform_paragraph_indents(entradas))

Saida:  
  ST_PARAGRAFO_UNICO, ST_ALINEA_A
  ST_PARAGRAFO_UNICO, ST_ALINEA_A, ST_ALINEA_B
  ST_PARAGRAFO_3, ST_ALINEA_A, ST_ALINEA_B, ST_ALINEA_C e ST_ALINEA_E, do art. 102
  art. 102, ST_PARAGRAFO_2, ST_ALINEA_A da Constituição Federal



# DATASET VICTOR

In [7]:
import pandas as pd

test = pd.read_csv('./data/train_small.csv')
text = test['body']

In [8]:
print("Dados de entrada:\n",text)

Dados de entrada:
 0         {"tribunal justiça estado bahia poder judiciár...
1         {"excelentíssimo senhor doutor juiz direito ju...
2         {"razões recurso inominado recorrente atlantic...
3         {"empresa recorrente tornou credora dos débito...
4         {"entretanto verdade parte apelante tornou tit...
                                ...                        
149212    {"supremo tribunal federal recurso extraordiná...
149213    {"seção recursos extraordinários mandado intim...
149214    {"ttar qsvòwi a edewrr seção recursos extraord...
149215    {"ertidao certifico dou que dirigi setor autar...
149216    {"supremo tribunal federal secretaria judiciár...
Name: body, Length: 149217, dtype: object


In [9]:
lista = []
for item in text:
    lista.append(transform_paragraph_indents(item))

In [10]:
print("Dados de saida: \n", lista[55])

Dados de saida: 
 {"superior tribunal justiça abono assiduidade licença prêmio não gozadas função necessidade serviço quais constituem verbas indenizatórias conforme está pacificado seio desta corte superior súmulas nºs juntada das declarações ajuste para fins verificação eventual compensação não estabelece fato constitutivo direito autor contrário perfazem fato extintivo seu direito cuja comprovação única exclusivamente parte fazenda nacional ocorrendo incidência fonte retenção indevida adicional imposto renda não necessidade se comprovar que responsável tributário recolheu respectiva importância aos cofres públicos não pode afastar pretensão restituição via precatório visto que contribuinte poderá escolher forma mais conveniente para pleitear execução decisão condenatória est por meio compensação restituição via precatório embargos acolhidos com parcial provimento recurso especial edcl resp relator ministro josé delgado de agravo regimental agravo instrumento tributário imposto renda